Task: analysis Airbnb data
<br/>
<img src="1.png" style="width:800px;height:480px;float:left">

In [1]:
import pandas as pd
import numpy as np

dc_listings = pd.read_csv('listings.csv')
print(dc_listings.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3723 entries, 0 to 3722
Data columns (total 92 columns):
id                                  3723 non-null int64
listing_url                         3723 non-null object
scrape_id                           3723 non-null int64
last_scraped                        3723 non-null object
name                                3723 non-null object
summary                             3457 non-null object
space                               2977 non-null object
description                         3722 non-null object
experiences_offered                 3723 non-null object
neighborhood_overview               2469 non-null object
notes                               1712 non-null object
transit                             2588 non-null object
thumbnail_url                       3690 non-null object
medium_url                          3690 non-null object
picture_url                         3723 non-null object
xl_picture_url                      3690

In [2]:
features = [
    'accommodates', 'bedrooms', 'bathrooms', 'beds', 'price', 'minimum_nights',
    'maximum_nights', 'number_of_reviews'
]
dc_listings = dc_listings[features]
print(dc_listings.shape)
dc_listings.head(10)

(3723, 8)


accommodates  bedrooms  bathrooms  beds    price  minimum_nights  \
0             4       1.0        1.0   2.0  $160.00               1   
1             6       3.0        3.0   3.0  $350.00               2   
2             1       1.0        2.0   1.0   $50.00               2   
3             2       1.0        1.0   1.0   $95.00               1   
4             4       1.0        1.0   1.0   $50.00               7   
5             4       2.0        1.0   4.0   $99.00               1   
6             4       2.0        2.0   2.0  $100.00               3   
7             2       1.0        1.0   1.0  $100.00               1   
8             2       1.0        1.5   1.0   $38.00               2   
9             2       1.0        NaN   1.0   $71.00               2   

   maximum_nights  number_of_reviews  
0            1125                  0  
1              30                 65  
2            1125                  1  
3            1125                  0  
4            1125                  0  
5            1125                  0  
6            1125                  0  
7            1125                  0  
8             180                  1  
9             365                  4

If I have a 3-bedrooms house, how much can I have?
(k-nearest neighbors: k - how many neeighbors are chosen to compare)

<img src="2.png" style="width:600px;height:230px;float:left">

#### The principle of KNN:
<img src="3.png" style="width:600px;height:330px;float:left"> <br/><br/>
<br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/>

##### comapre the similar data (bedroom =1) in data set:
<img src="4.png" style="width:600px;height:330px;float:left"><br/><br/>
<br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/>

##### if k=3, then:
<img src="5.png" style="width:600px;height:330px;float:left"><br/><br/>
<br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/>

That is the mean value of the rooms

#### Definition of the distance:
$$d=\sqrt{(q_{1} - p_{1} )^2 + (q_{2} - p_{2} )^2 + ... + (q_{n} - p_{n} )^2}$$
<br/>
measure the distance between P and Q

In [3]:
# assume we have a 3-bedroom house:
our_value = 3

# difference between accomodates
dc_listings['distance'] = np.abs(dc_listings.accommodates - our_value)  
dc_listings.distance.value_counts().sort_index()

#  0-> houses that also 3 accomodates
#  1-> houses that 5 or 4 accomodates, etc

0      461
1     2294
2      503
3      279
4       35
5       73
6       17
7       22
8        7
9       12
10       2
11       4
12       6
13       8
Name: distance, dtype: int64

In [4]:
# Shuffle the dataframe
dc_listings = dc_listings.sample(frac=1,random_state=0)
# sort the list according to the distance
dc_listings = dc_listings.sort_values('distance')
print(dc_listings.price.head(10))

2645     $75.00
2825    $120.00
2145     $90.00
2541     $50.00
3349    $105.00
2228    $117.00
1122    $176.00
2311    $117.00
625     $150.00
2312    $100.00
Name: price, dtype: object


In [5]:
# remove the '$" and convert the data type from string to float
dc_listings['price'] = dc_listings['price'].str.replace("\$|,",'').astype(float)
# the mean value of 5-nearest neighbors
mean_price = dc_listings.price[:5].mean()
print('the random 5-nearest mean value:',mean_price)

mean_price = dc_listings.price[dc_listings['distance']==0].mean()
print('the global mean value:',mean_price)

the random 5-nearest mean value: 88.0
the global mean value: 146.1691973969631


### Model Evaluation
<img src="7.png" style="width:600px;height:250px;float:left">
<br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/>

using training set and test set:<br/>
training set is using to guess the predict value, the test set is using to prove the guessing.

In [6]:
features = [
    'accommodates', 'bedrooms', 'bathrooms', 'beds', 'price', 'minimum_nights',
    'maximum_nights', 'number_of_reviews'
]
dc_listings = pd.read_csv('listings.csv')[features]
dc_listings['price'] = dc_listings['price'].str.replace("\$|,",'').astype(float)

train_df = dc_listings.copy().iloc[:2792]
test_df = dc_listings.copy().iloc[2792:]

In [7]:
# mean value of KNN_5
def predict_price(new_listing_value, feature_column):
    temp_df = train_df
    temp_df['difference'] = np.abs(
        dc_listings[feature_column] - new_listing_value)  #difference
    temp_df = temp_df.sort_values(
        'difference')  #sort according to the difference
    knn_5 = temp_df.price.iloc[:5]
    predicted_price = knn_5.mean()  #mean value of 5 nearest ones
    return (predicted_price)

In [1]:
# Series.apply test
import pandas as pd
x = pd.DataFrame([1, 2, 3, 4])

def add(i1, i2):
    return i1 + i2

x.apply(add, i2=5)

# The "add" function has two parameters: i1, i2.
# The first parameter is going to be the value in data
# frame and the second is whatever we pass to the "apply"
# function. In this case, we are passing "5" to the apply
# function using the keyword argument "i2".

0
0  6
1  7
2  8
3  9

In [9]:
test_df['predicted_price'] = test_df.accommodates.apply(
    predict_price, feature_column='accommodates')
test_df.head()

accommodates  bedrooms  bathrooms  beds  price  minimum_nights  \
2792             2       0.0        1.0   1.0  120.0               1   
2793             3       2.0        2.0   1.0  140.0               2   
2794             4       2.0        1.0   1.0  299.0               2   
2795             3       1.0        1.0   1.0   85.0               1   
2796             6       2.0        2.0   3.0  175.0               1   

      maximum_nights  number_of_reviews  predicted_price  
2792            1125                  8            104.0  
2793            1125                  7            177.4  
2794            1125                  5            145.8  
2795              92                  2            177.4  
2796            1125                 62            187.2

#### root mean squared error (RMSE) <br/>
$$ RMSE = \sqrt{\frac{(actual_{1} - predicted_{1})^2 + (actual_{2} - predicted_{2})^2 + ... + (actual_{n} - predicted_{n})^2}{n}} $$

In [10]:
test_df['squared_error'] = (test_df['predicted_price'] - test_df['price'])**(2)
mse = test_df['squared_error'].mean()
rmse = mse ** (1/2)
rmse

136.55228072269364

In [12]:
for feature in ['accommodates','bedrooms','bathrooms','number_of_reviews']:
    #test_df['predicted_price'] = test_df.accommodates.apply(predict_price,feature_column=feature)
    test_df['predicted_price'] = test_df[feature].apply(predict_price,feature_column=feature)
    test_df['squared_error'] = (test_df['predicted_price'] - test_df['price'])**(2)
    mse = test_df['squared_error'].mean()
    rmse = mse ** (1/2)
    print("RMSE for the {} column: {}".format(feature,rmse))

RMSE for the accommodates column: 136.55228072269364
RMSE for the bedrooms column: 125.71548444404552
RMSE for the bathrooms column: 135.6666653295221
RMSE for the number_of_reviews column: 151.96713676058198


In [13]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
features = ['accommodates','bedrooms','bathrooms','beds','price','minimum_nights','maximum_nights','number_of_reviews']

dc_listings = pd.read_csv('listings.csv')
dc_listings = dc_listings[features]
dc_listings['price'] = dc_listings.price.str.replace("\$|,",'').astype(float)
dc_listings = dc_listings.dropna()

dc_listings[features] = StandardScaler().fit_transform(dc_listings[features])
normalized_listings = dc_listings

print(dc_listings.shape)
normalized_listings.head()

(3671, 8)


accommodates  bedrooms  bathrooms      beds     price  minimum_nights  \
0      0.401420 -0.249501  -0.439211  0.297386  0.081119       -0.341421   
1      1.399466  2.129508   2.969551  1.141704  1.462622       -0.065047   
2     -1.095648 -0.249501   1.265170 -0.546933 -0.718699       -0.065047   
3     -0.596625 -0.249501  -0.439211 -0.546933 -0.391501       -0.341421   
4      0.401420 -0.249501  -0.439211 -0.546933 -0.718699        1.316824   

   maximum_nights  number_of_reviews  
0       -0.016575          -0.516779  
1       -0.016606           1.706767  
2       -0.016575          -0.482571  
3       -0.016575          -0.516779  
4       -0.016575          -0.516779

In [14]:
norm_train_df = normalized_listings.copy().iloc[0:2792]
norm_test_df = normalized_listings.copy().iloc[2792:]